In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os



# Import API key
from api_keys import g_key

In [29]:
file = "output/true_combined.csv"

county_df = pd.read_csv(file)
county_df.head()

file2 = "Resources/countycoordinates.xlsx"

coord_df = pd.read_excel(file2)
clean_coord = coord_df[["State", "County", "Latitude", "Longitude"]]
clean_coord

data_loc = pd.DataFrame.merge(county_df, clean_coord, on=["State", "County"])



data_loc.to_csv('output/clean_with_lat_long.csv', index = False)

data_loc

,State,County,POP2010,OHU2010,PovertyRate,MedianFamilyIncome,LAPOP1_10,LAPOP05_10,LAPOP1_20,LALOWI1_10,...,# Deaths,Years of Potential Life Lost Rate,% Obese,% Fair/Poor,# Diabetics,# Violent Crimes,# Unemployed,% Children in Poverty,Latitude,Longitude
0,Alabama,Autauga,54571,20221,13.858333,61082.916667,18092.661348,30141.638549,11064.872922,6835.642497,...,830.0,9158.0,34,18,711.0,145.0,1327.0,19.0,32.534920,-86.642749
1,Alabama,Baldwin,182265,73180,14.358065,60664.935484,46400.350641,72307.849152,43486.806226,15459.848416,...,2573.0,7394.0,27,16,2615.0,415.0,4791.0,20.0,30.660970,-87.749840
2,Alabama,Barbour,27457,9820,24.533333,43123.000000,6683.975691,9302.717960,1182.176872,3251.362574,...,413.0,8573.0,44,25,525.0,85.0,771.0,45.0,31.869603,-85.393197
3,Alabama,Bibb,22915,7953,16.025000,43362.500000,295.614426,295.614426,0.000000,164.903295,...,407.0,11766.0,40,20,333.0,33.0,561.0,29.0,32.998644,-87.126439
4,Alabama,Blount,57322,21578,17.911111,52136.000000,5855.939161,7387.084978,3900.420369,2570.900481,...,906.0,9333.0,35,19,705.0,123.0,1316.0,22.0,33.980867,-86.567371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3090,Wyoming,Sweetwater,43806,16475,11.841667,81835.083333,9400.240856,16730.468695,8914.971285,1816.028671,...,445.0,6878.0,32,16,469.0,159.0,1053.0,10.0,41.659496,-108.879431
3091,Wyoming,Teton,21294,8973,8.100000,89555.250000,3711.377493,3949.327991,2605.093461,972.594547,...,116.0,3728.0,13,12,104.0,NaN,558.0,8.0,43.934637,-110.589810
3092,Wyoming,Uinta,21118,7668,14.166667,65312.666667,758.694309,758.694309,13.350968,318.523395,...,234.0,7660.0,28,15,243.0,8.0,475.0,12.0,41.287492,-110.547562
3093,Wyoming,Washakie,8533,3492,14.133333,62213.666667,1057.246483,2537.470166,885.406917,359.634002,...,97.0,7294.0,28,14,156.0,7.0,174.0,16.0,43.904997,-107.682861


In [39]:
# Plot Heatmap
fig = gmaps.figure()

obese = data_loc["% Obese"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=obese, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

#Display figure
fig


Figure(layout=FigureLayout(height='420px'))